# Assignment #1
John-Craig Borman

3/13/19

QF 302

## Problem 1

> Collect the bid/ask spread of Netflix's stock price using the SHIFT system with $T = 120$ and $\Delta t = 5$ seconds:

In [57]:
import shift
import time
import pandas as pd

debug = True

In [58]:
trader = shift.Trader("jborman")
trader.connect("initiator.cfg", "hR7fgSbU")

True

In [59]:
trader.subAllOrderBook()

True

In [60]:
def get_data(ticker, interval=120, wait=5, trader=trader):
    out = pd.DataFrame(columns=["Best", "Bid", "Ask", "bVolume", "aVolume"])
    
    for i in range(interval):
        if debug: print("loop {}".format(i))
            
        bp = trader.getBestPrice(ticker)
            
        out.loc[i] = [
            bp.,
            bp.getGlobalBidPrice(), bp.getGlobalAskPrice(),
            bp.getGlobalBidSize(), bp.getGlobalAskSize()
        ]
        
        if i == interval - 1:
            break
        else:
            time.sleep(wait)
            
    return out

In [61]:
df = get_data("NFLX")
df.head()

loop 0


AttributeError: 'shift.BestPrice' object has no attribute 'getBestPrice'


### 1.a

**Present a table of summary statistics of $A_t$ and $B_t$**


In [56]:
df[["Bid", "Ask"]].describe()

,Bid,Ask
count,1.200000e+02,1.200000e+02
mean,3.366200e+02,3.369500e+02
std,1.712453e-13,1.141635e-13
min,3.366200e+02,3.369500e+02
25%,3.366200e+02,3.369500e+02
50%,3.366200e+02,3.369500e+02
75%,3.366200e+02,3.369500e+02
max,3.366200e+02,3.369500e+02



### 1.b

**Calculate $S^Q$, $S^E$ and $ILLIQ$ for the same data. Plot all three measures in a single graph with two y-axes.**

$$ S^Q = \frac{1}{T} \sum_{t=1}^{T}(A_t - B_t) $$

$$ S^E = \frac{1}{T} \sum_{t=1}^{T}(P_t - M_t); \quad M_t = \frac{1}{2}(A_t + B_t)$$
    
$$ ILLIQ = \frac{1}{T} \sum_{k=1}^{T} \frac{|r_k|}{V_k} $$

TypeError: 'shift.BestPrice' object is not callable